# Clase 2 - Contestar a preguntas sencillas sobre datos

## Objetivos

Utilizar datos reales y crear código para contestar preguntas sobre el conjunto.

### Pasos

1. leer el input (1 json per línea)
2. modelar cada línea correcta en una data class que represente a un Tweet 
3. contestar preguntas sobre el conjunto de datos obtenidos

## Repaso de Python: I/O con ficheros

### Escribir ficheros

El siguiente snippet de código muestra como escribir un fichero (observen la `w` - de *write* - como segundo argumento del método `open`)
Además, observen el carater espeecial `\n` para insertar un final de línea. 


```python
with open('/some/path/to/a/file', 'w') as f:
  f.writeline("a line\n")
```

In [2]:
with open('/tmp/test-file.txt', 'w') as input:
  input.write("test1\n")
  input.write("test2\n")
  input.writelines(["test3\n", 'test4\n'])


### Leer ficheros

El siguiente snippet de código muestra como leer un fichero (observen la `r` - de *read* - como segundo argumento del método `open`)


```python
with open('/some/path/to/a/file', 'r') as f:
  line = f.readline()
```

### Leer linea por línea

Es la manera recomendada en leer ficheros grandes en Python, para evitar de mantener todo el fichero en memoria, que es un recurso escaso. 

In [3]:
with open('/tmp/test-file.txt', 'r') as f:
    for line in f:
      print(line.rstrip())

test1
test2
test3
test4


### Leer solo las primeras N líneas

Puede ser útil para explorar los ficheros de input y hacerse una idea de la forma de los datos 

In [5]:
with open('/tmp/test-file.txt', 'r') as f:
    ## Lee cada línea (next) para cada elemento de una lista de 0 a N y añadela a la lista head
    head = [next(f).rstrip() for _ in range(3)]
    ## Imprime una línea por cada línea en head (que es una lista) 
    [print(line) for line in head]


test1
test2
test3


# ETL (Extract, transform, Load)




### Estudiar el input

Descargamos el fichero `mini_input.txt` y lo subimos a nuestro entorno Jupyter. Procedemos a explorarlo, leendo las primeras líneas del mismo. 

In [6]:
with open('mini_input.txt', 'r') as f:
    head = [next(f).rstrip() for _ in range(3)]
    [print(line) for line in head]


{"created_at":"Sat May 12 15:58:53 +0000 2018","id":995332494974210048,"id_str":"995332494974210048","text":"RT @carloscarmo98: -Manel, algo que decir sobre tu actuaci\u00f3n en Eurovision?\n-Kikiriketediga https:\/\/t.co\/yXGYtKmJoM","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":492271155,"id_str":"492271155","name":"alba aguirre","screen_name":"Alba137","location":"en pleno akelarre","url":null,"description":"no todo lo que brilla es oro, a veces es highlight \u2728\ud83d\udc9c","translator_type":"regular","protected":false,"verified":false,"followers_count":718,"friends_count":416,"listed_count":2,"favourites_count":24718,"statuses_count":21764,"created_at":"Tue Feb 14 14:46:34 +0000 2012","utc_offset":10800,"time_zone":"Athen

## Modelizar Tweets

Representar un Tweet dentro de una dataclass de python. Desde el input de arriba sabemos que: 
- el input es en formato JSON 
- contiene muchos campos, por lo tanto seleccionaremos algunos relevantes, u ignoraremos los otros

In [8]:
from dataclasses import dataclass

@dataclass
class Tweet:
  """Class to model a Tweet"""
  id: int         # The unique ID of a tweet
  content: str    # The textual content of a tweet
  author: str     # The nickname of the author of the tweet
  language: str   # The language of the tweet


En el código siguiente, el método `parse_line` se ocupa de interpretar una línea de input como JSON, y de mapear cada línea en una instancia de la dataclass `Tweet`. 

De momento, aunque no se recomiende para inputs de gran tamaño, almacenaremos **todos** los Tweets en una lista `tweets`, que escribiremos en un fichero `clean-dataset`. 

Completamos juntos *en clase* el código del método `parse_line`. 

In [10]:
import json, dataclasses

tweets = []

def parse_line(line: str):
  """Try to parse a string into a Person"""
  try:
    parsed = json.loads(line)
    return Tweet(parsed['id'], parsed['text'], parsed['user']['screen_name'], parsed['lang'])
  except Exception as e:
    print(f"Error parsing '{line}': {e}")

with open("mini_input.txt") as input:
    for line in input:
        if len(line.rstrip()) > 0:
          tweet = parse_line(line)
          tweets.append(tweet)

#print(tweets)

with open("clean-dataset", 'w') as f:
 tweet_strings = map(lambda x: json.dumps(dataclasses.asdict(x)) + '\n', tweets)
 f.writelines(tweet_strings)


In [ ]:
for modeled_tweet in tweets[0:10]:
  print(modeled_tweet)

# Responder preguntas sobre datos

Desde nuestro dataset limpio `clean_input` podemos ahora intentar responder a preguntas concretas, por ejemplo:

1. Cuántos tweets hay en nuestro fichero `mini_input.txt`? 
2. Cuántos de estos tweets son en español? Y cuantos son en inglés? 
3. Cual es la palabra más significativa de los tweets en castellano? 

In [21]:
import json, dataclasses

def read_clean_tweets(input: str):
  tweets = []
  with open(input, 'r') as f:
    lines = f.readlines()
  for line in lines:
    parsed = json.loads(line)
    tweet = Tweet(**parsed)
    tweets.append(tweet)
  return tweets

def count_tweets(tweets: list[Tweet]):
    return len(tweets)

def count_spanish_tweets(tweets: list[Tweet]):
    count = 0 
    for tweet in tweets:
        if tweet.language == 'es':
            count += 1
    return count 

def most_significant_word_in_lang(tweets: list[Tweet], lang: str):
    counts = {} 
    for tweet in tweets:
        if tweet.language == lang: 
            for word in tweet.content.split(' '):
                if word in counts: 
                    current_value = counts[word]
                    new_value = current_value + 1
                    counts[word] = new_value
                else:
                    counts[word] = 1
    return dict(sorted(counts.items(), key = lambda x: x[1], reverse = True))
            

tweets = read_clean_tweets('clean-dataset')

print(tweets[0:2])

tweets_count = count_tweets(tweets)

spanish_tweets_count = count_spanish_tweets(tweets)

most_significant_word_in_spanish = most_significant_word_in_lang(tweets, 'it')

print(tweets_count)

print(spanish_tweets_count)

print(most_significant_word_in_spanish)

[Tweet(id=995332494974210048, content='RT @carloscarmo98: -Manel, algo que decir sobre tu actuación en Eurovision?\n-Kikiriketediga https://t.co/yXGYtKmJoM', author='Alba137', language='es'), Tweet(id=995332495783727105, content="RT @DougJ7777: If Britain wins #Eurovision then we have to rejoin the EU. It's in the rules. #Eurovision2018", author='DougJ7777', language='en')]
1000
236
{'RT': 17, 'e': 12, '-': 9, '@MetaErmal': 8, 'in': 8, 'il': 7, '@FabrizioMoroOff': 6, 'di': 6, 'a': 5, 'Song': 5, 'Contest': 5, 'Eurovision': 5, 'che': 5, 'per': 5, '#Eurovision': 5, 'è': 4, '#ESC2018': 4, 'Stasera': 4, 'la': 4, 'con': 4, 'si': 3, 'sera': 3, 'ci': 3, 'non': 3, '#MetaMoro': 3, 'Ermal': 3, '#ESCITA': 2, 'poche': 2, 'ore': 2, 'delle': 2, 'gara': 2, 'c…': 2, 'può': 2, 'ht…': 2, 'gli': 2, '@ElisaDospina:': 2, 'Si': 2, 'le': 2, 'più': 2, 'Meta': 2, 'Fabrizio': 2, 'della': 2, 'questo': 2, 'video': 2, 'DEL': 2, 'vi': 2, 'tv': 2, '12': 2, 'maggio': 2, '2018': 2, 'Amici,': 2, 'Ulisse': 2, 'Il': 2, '@

## Brainstorming

What other questions could we ask to the data?

# Question 2.1

Cuántos tweets son originales? 

Definimos un Tweet como *original* si no es un retweet. Para nuestros datos, diremos que un tweet es un retweet si contiene el campo `retweeted_status`. 

Pasos a seguir: 
- Anadir un elemento a nuestra dataclass Tweet (que tipo mejor representa esta información?) 
- Leer de nuevo el input teniendo en cuenta este nuevo elemento y el criterio establecido arriba para determinar si un tweet as un retweet
- Escribir un método `count_original_tweets` que devuelva el recuento de tweet originales